In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from src.bayesian_network import BayesianNetwork


In [19]:
import pandas as pd

# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
data = pd.read_csv(url, names=columns)

from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [20]:
categorical_columns = ['Outcome']
bn = BayesianNetwork(method='hill_climb', max_parents=3, categorical_columns=categorical_columns)

try:
    # Fit the model
    bn.fit(train_data)

    # Evaluate the model
    test_ll = bn.log_likelihood(test_data)
    print(f"\nTest log-likelihood: {test_ll:.4f}")

    # Cross-validation
    mean_ll, std_ll = bn.cross_validate(data, k_folds=5)
    print(f"Cross-validation: Mean LL = {mean_ll:.4f}, Std = {std_ll:.4f}")

    # Compute sensitivity for the 'Outcome' variable
    sensitivity = bn.compute_sensitivity('Outcome', num_samples=1000)
    print("\nSensitivity analysis for 'Outcome':")
    for node, value in sorted(sensitivity.items(), key=lambda x: x[1], reverse=True):
        print(f"Sensitivity to {node}: {value:.4f}")

    # Network structure
    print("\nNetwork structure:")
    if hasattr(bn, 'explain_structure'):
        structure = bn.explain_structure()
        for node, parents in structure.items():
            print(f"{node}: {parents}")
    else:
        print("explain_structure method is not implemented")

except AttributeError as e:
    print(f"AttributeError: {e}")
    print("It seems that some methods are not implemented in the BayesianNetwork class.")
    print("Please ensure all required methods are properly implemented.")

except Exception as e:
    print(f"An error occurred: {e}")

TypeError: BayesianNetwork.__init__() got an unexpected keyword argument 'categorical_columns'